# Clean Census Block Data
Step required to make the Probabilistic Housing Unit Allocation work.

Census Block Data needs to be combined with Census Place and PUMA data.

Initial Block Data provides state, county, tract, and block group information but does not identify the Census Place (City) or the PUMA (Public Use Microdata Area). 

Census Blocks are used to define both Place Boundaries and PUMA boundaries. The geographies should be "nested" without any overlap between polygons.
    

## Description of Program
- program:    IN-CORE_1av2_Lumberton_CleanBlockData
- task:       Prepare Census Block Data for Labor Market Allocation
- Version:    2021-06-02
- project:    Interdependent Networked Community Resilience Modeling Environment (IN-CORE) Subtask 5.2 - Social Institutions
- funding:	  NIST Financial Assistance Award Numbers: 70NANB15H044 and 70NANB20H008 
- author:     Nathanael Rosenheim

- Suggested Citation:
Rosenheim, N. (2021) “Obtain, Clean, and Explore Labor Market Allocation Methods". 
Archived on Github and ICPSR.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import math as math
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely
import descartes

import folium as fm # folium has more dynamic maps - but requires internet connection

import os # For saving output to path

In [2]:
# Display versions being used - important information for replication
import sys
print("Python Version     ", sys.version)
print("numpy version:     ", np.__version__)
print("geopandas version: ", gpd.__version__)
print("pandas version:    ", pd.__version__)
print("shapely version:   ", shapely.__version__)
# print("descartes version:   ", descartes.__version__)  1.1.0
print("folium version:    ", fm.__version__)

Python Version      3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 15:37:01) [MSC v.1916 64 bit (AMD64)]
numpy version:      1.20.2
geopandas version:  0.9.0
pandas version:     1.2.4
shapely version:    1.7.1
folium version:     0.12.1


In [3]:
# Store Program Name for output files to have the same name
programname = "IN-CORE_1av2_Lumberton_CleanBlockData_2021-06-02"
# Make directory to save output
if not os.path.exists(programname):
    os.mkdir(programname)

## Read in Census Block Data
Census Blocks provide an estimate of how many residiential address points (housing units) should be located in each block.

In [4]:
# Read data from www2.census.gov for 2010 Census Block Data with Population and Housing Unit Counts
# Data is a zipped Shapefile with all census blocks for 1 state. The State FIPS code must be changed
census_blocks_shp = 'https://www2.census.gov/geo/tiger/TIGER2010BLKPOPHU/tabblock2010_37_pophu.zip'
census_blocks_gdf = gpd.read_file(census_blocks_shp)
census_blocks_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry
0,37,001,021801,2010,370010218012010,N,0,0,"POLYGON ((-79.48485 35.99757, -79.48488 35.997..."
1,37,001,021801,2013,370010218012013,N,0,0,"POLYGON ((-79.47881 35.99998, -79.47897 35.999..."
2,37,001,021801,2009,370010218012009,N,0,0,"POLYGON ((-79.47414 36.00568, -79.47408 36.005..."
3,37,001,021801,2001,370010218012001,N,0,0,"POLYGON ((-79.46624 36.00216, -79.46630 36.002..."
4,37,001,021206,3007,370010212063007,N,0,0,"POLYGON ((-79.27035 36.09526, -79.27052 36.095..."


In [5]:
census_blocks_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

#### Note 
EPSG 4269 uses NAD 83 which will have slightly different lat lon points when compared to EPSG 4326 which uses WGS 84.

In [6]:
# Select Counties for Robeson County (37155)
countyselect = ["155"]
census_blocks_gdf['CountySelect'] = np.where(census_blocks_gdf['COUNTYFP10'].isin(countyselect),1,0)
census_blocks_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,CountySelect
0,37,001,021801,2010,370010218012010,N,0,0,"POLYGON ((-79.48485 35.99757, -79.48488 35.997...",0
1,37,001,021801,2013,370010218012013,N,0,0,"POLYGON ((-79.47881 35.99998, -79.47897 35.999...",0
2,37,001,021801,2009,370010218012009,N,0,0,"POLYGON ((-79.47414 36.00568, -79.47408 36.005...",0
3,37,001,021801,2001,370010218012001,N,0,0,"POLYGON ((-79.46624 36.00216, -79.46630 36.002...",0
4,37,001,021206,3007,370010212063007,N,0,0,"POLYGON ((-79.27035 36.09526, -79.27052 36.095...",0


In [7]:
census_blocks_37155_gdf = census_blocks_gdf[census_blocks_gdf['CountySelect'] == 1]
census_blocks_37155_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,CountySelect
221421,37,155,961900,2028,371559619002028,N,14,52,"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",1
221422,37,155,961900,2054,371559619002054,N,1,3,"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",1
221423,37,155,961700,2069,371559617002069,N,41,99,"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",1
221424,37,155,961700,2065,371559617002065,N,6,22,"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",1
221425,37,155,961700,2058,371559617002058,N,19,55,"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",1


In [8]:
# Add Representative Point
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index, 'rppnt4269'] = census_blocks_37155_gdf['geometry'].representative_point()
census_blocks_37155_gdf['rppnt4269'].label = "Representative Point EPSG 4269 (WKT)"
census_blocks_37155_gdf['rppnt4269'].notes = "Internal Point within census block poly EPSG 4269"

# Add Column that Duplicates Polygon Geometry - allows for swithcing between point and polygon geometries for spatial join
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index, 'blk104269'] = census_blocks_37155_gdf['geometry']
census_blocks_37155_gdf['blk104269'].label = "2010 Census Block Polygon EPSG 4269 (WKT)"
census_blocks_37155_gdf['blk104269'].notes = "Polygon Shape Points for 2010 Census Block EPSG 4269"

C:\Users\nathanael99\Miniconda3\envs\pyincoreEnv20210506\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
C:\Users\nathanael99\Miniconda3\envs\pyincoreEnv20210506\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Add Address Point Count that Includes Group Quarters

In [9]:
census_addresspoints = 'IN-CORE_2bv1_37155_BlockAPCounts_2016-10-27.csv'
census_addresspoints = pd.read_csv(census_addresspoints)
census_addresspoints.head()

,blockid,tothupoints,popcount,HU100,POP100
0,371559601011000,0,0,0,0
1,371559601011001,0,0,0,0
2,371559601011002,1,2,1,2
3,371559601011003,3,6,3,6
4,371559601011004,0,0,0,0


In [10]:
# Merge ID - Block ID - Needs to be a string
census_addresspoints['blockid'].dtype

dtype('int64')

In [11]:
# Convert blockid Parcel ID to a String
census_addresspoints['BLOCKID10'] = census_addresspoints['blockid'].apply(lambda x : str((x)))
census_addresspoints['BLOCKID10'].dtype

dtype('O')

In [12]:
census_blocks_37155_gdf['BLOCKID10'].dtype

dtype('O')

In [13]:
# Merge Address Point Count with Block Data 
census_blocks_37155_gdf = pd.merge(census_blocks_37155_gdf, census_addresspoints,
                                  left_on='BLOCKID10', right_on='BLOCKID10', how='left')
census_blocks_37155_gdf.BLOCKID10.describe()

count                5799
unique               5799
top       371559613024004
freq                    1
Name: BLOCKID10, dtype: object

In [14]:
census_blocks_37155_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,CountySelect,rppnt4269,blk104269,blockid,tothupoints,popcount,HU100,POP100
0,37,155,961900,2028,371559619002028,N,14,52,"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",1,POINT (-79.22459 34.45879),"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",371559619002028,14,51,14,52
1,37,155,961900,2054,371559619002054,N,1,3,"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",1,POINT (-79.18141 34.40608),"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",371559619002054,1,3,1,3
2,37,155,961700,2069,371559617002069,N,41,99,"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",1,POINT (-79.16202 34.48769),"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",371559617002069,41,99,41,99
3,37,155,961700,2065,371559617002065,N,6,22,"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",1,POINT (-79.16260 34.50130),"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",371559617002065,6,21,6,22
4,37,155,961700,2058,371559617002058,N,19,55,"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",1,POINT (-79.14702 34.49788),"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",371559617002058,19,55,19,55


In [15]:
# Compare Population Counts - they should be equal - differences come from Households with more than 7 people or Group Quarters
census_blocks_37155_gdf['popdiff'] = census_blocks_37155_gdf['POP10'] - census_blocks_37155_gdf['popcount']
census_blocks_37155_gdf['popdiff'].describe()

count    5799.000000
mean        0.167098
std         1.303343
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        70.000000
Name: popdiff, dtype: float64

## Add Place Name (Cities) To Blocks
### Read in place polygons for state and select places in study area
Place names provide link to population demographics for cities and places defined by the Census. The Census communicates with cities and updates city boundaries based on policitical boundaries set by communities.

In [16]:
# Location of Place Names Defined By US Census - data is available by state
census_place_shp = 'https://www2.census.gov/geo/tiger/TIGER2010/PLACE/2010/tl_2010_37_place10.zip'
census_place_gdf = gpd.read_file(census_place_shp)
census_place_gdf.head()

,STATEFP10,PLACEFP10,PLACENS10,GEOID10,NAME10,NAMELSAD10,LSAD10,CLASSFP10,PCICBSA10,PCINECTA10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,37,74760,02406902,3774760,Wingate,Wingate town,43,C1,N,N,G4110,A,5152353,12812,+34.9848492,-080.4499498,"MULTIPOLYGON (((-80.42983 35.00070, -80.42937 ..."
1,37,71940,02407569,3771940,Wesley Chapel,Wesley Chapel village,47,C1,N,N,G4110,A,24546920,226993,+35.0058569,-080.6934046,"MULTIPOLYGON (((-80.67979 34.99356, -80.67964 ..."
2,37,71460,02406844,3771460,Waxhaw,Waxhaw town,43,C1,N,N,G4110,A,29899550,309769,+34.9378871,-080.7379328,"MULTIPOLYGON (((-80.72146 34.90322, -80.72135 ..."
3,37,69260,02406780,3769260,Unionville,Unionville town,43,C1,N,N,G4110,A,69821759,629943,+35.0741443,-080.5200404,"POLYGON ((-80.52167 35.03138, -80.52230 35.031..."
4,37,43920,02404284,3743920,Monroe,Monroe city,25,C1,N,N,G4110,A,77067720,1625835,+35.0015898,-080.5612792,"MULTIPOLYGON (((-80.48048 35.03684, -80.47923 ..."


In [17]:
census_place_gdf['PLACEFP10'].describe()

count       739
unique      739
top       01640
freq          1
Name: PLACEFP10, dtype: object

In [18]:
census_place_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [19]:
census_blocks_37155_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [20]:
# Find the bounds of the Census Block File
minx = census_blocks_37155_gdf.bounds.minx.min()
miny = census_blocks_37155_gdf.bounds.miny.min()
maxx = census_blocks_37155_gdf.bounds.maxx.max()
maxy = census_blocks_37155_gdf.bounds.maxy.max()
census_blocks_37155_gdf_bounds = [minx, miny, maxx, maxy]
census_blocks_37155_gdf_bounds

[-79.461754, 34.29924, -78.805144, 34.953637]

In [21]:
# Select Places within Bounds of Study Area
# build the r-tree index - for Places
sindex_census_place_gdf = census_place_gdf.sindex
possible_matches_index = list(sindex_census_place_gdf.intersection(census_blocks_37155_gdf_bounds))
area_census_place_gdf = census_place_gdf.iloc[possible_matches_index]
area_census_place_gdf['NAME10'].describe()

count            36
unique           36
top       Chadbourn
freq              1
Name: NAME10, dtype: object

In [22]:
# plot the intersections and the city
census_place_gdf_map = fm.Map(location=[(miny+maxy)/2,(minx+maxx)/2], zoom_start=10)
fm.GeoJson(area_census_place_gdf).add_to(census_place_gdf_map)
display(census_place_gdf_map)

### Spatial Join Place Names to Block IDS

In [23]:
# Confirm Count of Unique ID in layer to which data will be added
census_blocks_37155_gdf['BLOCKID10'].describe()

count                5799
unique               5799
top       371559613024004
freq                    1
Name: BLOCKID10, dtype: object

In [24]:
# build the r-tree index - Using Representative Point
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index,'geometry'] = census_blocks_37155_gdf['rppnt4269']
sindex_census_blocks_37155_gdf = census_blocks_37155_gdf.sindex

# find the points that intersect with each subpolygon and add ID to Point
for index, place in area_census_place_gdf.iterrows():
    # print(place['NAME10'])

    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex_census_blocks_37155_gdf.intersection(place['geometry'].bounds))
    possible_matches = census_blocks_37155_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(place['geometry'])]
    census_blocks_37155_gdf.loc[precise_matches.index,'PLCGEOID10'] = place['GEOID10']
    census_blocks_37155_gdf.loc[precise_matches.index,'PLCNAME10'] = place['NAME10']

In [25]:
# Confirm Count of Unique ID in layer to which data will be added
census_blocks_37155_gdf['BLOCKID10'].describe()

count                5799
unique               5799
top       371559613024004
freq                    1
Name: BLOCKID10, dtype: object

In [26]:
census_blocks_37155_gdf['PLCGEOID10'].describe()

count        2106
unique         22
top       3739700
freq         1047
Name: PLCGEOID10, dtype: object

In [27]:
# Switch Block Geography back to polygons
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index,'geometry'] = census_blocks_37155_gdf['blk104269']
census_blocks_37155_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,CountySelect,rppnt4269,blk104269,blockid,tothupoints,popcount,HU100,POP100,popdiff,PLCGEOID10,PLCNAME10
0,37,155,961900,2028,371559619002028,N,14,52,"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",1,POINT (-79.22459 34.45879),"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",371559619002028,14,51,14,52,1,NaN,NaN
1,37,155,961900,2054,371559619002054,N,1,3,"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",1,POINT (-79.18141 34.40608),"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",371559619002054,1,3,1,3,0,NaN,NaN
2,37,155,961700,2069,371559617002069,N,41,99,"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",1,POINT (-79.16202 34.48769),"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",371559617002069,41,99,41,99,0,NaN,NaN
3,37,155,961700,2065,371559617002065,N,6,22,"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",1,POINT (-79.16260 34.50130),"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",371559617002065,6,21,6,22,1,NaN,NaN
4,37,155,961700,2058,371559617002058,N,19,55,"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",1,POINT (-79.14702 34.49788),"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",371559617002058,19,55,19,55,0,NaN,NaN


In [28]:
# Look at One Place plot the intersections and the city
place_gdf_map = fm.Map(location=[(miny+maxy)/2,(minx+maxx)/2], zoom_start=10)
census_blocks_select_gdf = census_blocks_37155_gdf[census_blocks_37155_gdf['PLCNAME10'].notnull()]
blockstyle_function = lambda x: {'color':'green','fillColor': 'transparent' }
placetooltip=fm.features.GeoJsonTooltip(fields=['NAME10'],
                                              aliases = ['Place Name'],
                                              labels=True,
                                              sticky=False
                                             )
fm.GeoJson(census_blocks_select_gdf['geometry'],name='Census Blocks',style_function=blockstyle_function).add_to(place_gdf_map)
fm.GeoJson(area_census_place_gdf,name='Census Places',tooltip=placetooltip).add_to(place_gdf_map)
fm.LayerControl().add_to(place_gdf_map)
place_gdf_map.save(programname+'census_blocks_places.html')
# Error Displaying Map display(neosho_place_gdf_map)

## How many blocks do not have place names?

In [29]:
census_blocks_37155_gdf[census_blocks_37155_gdf.PLCNAME10.isnull()].blockid.describe()

count    3.693000e+03
mean     3.715596e+14
std      6.843916e+06
min      3.715596e+14
25%      3.715596e+14
50%      3.715596e+14
75%      3.715596e+14
max      3.715596e+14
Name: blockid, dtype: float64

## How many places do not have blocks?

In [30]:
# Collapse Blocks By Place Name and Count Blocks 
census_blocks_gdf_blockcount = census_blocks_37155_gdf[['PLCNAME10']]
census_blocks_gdf_blockcount['block_count'] = 1
census_blocks_gdf_blockcount_sum = census_blocks_gdf_blockcount.groupby(['PLCNAME10']).sum()
census_blocks_gdf_blockcount_sum.head()

C:\Users\nathanael99\Miniconda3\envs\pyincoreEnv20210506\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,block_count
PLCNAME10,
Barker Ten Mile,37
Elrod,10
Fairmont,146
Lumber Bridge,18
Lumberton,1047


In [31]:
# Add Block Count to Place Data
area_census_place_gdf_checkcount = pd.merge(area_census_place_gdf, census_blocks_gdf_blockcount_sum,
                                  left_on='NAME10', right_on='PLCNAME10', how='left')
area_census_place_gdf_checkcount.loc[area_census_place_gdf_checkcount['block_count'].isnull()]

,STATEFP10,PLACEFP10,PLACENS10,GEOID10,NAME10,NAMELSAD10,LSAD10,CLASSFP10,PCICBSA10,PCINECTA10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,block_count
0,37,22240,02406477,3722240,Fair Bluff,Fair Bluff town,43,C1,N,N,G4110,A,5555591,0,+34.3109588,-079.0343300,"POLYGON ((-79.04676 34.30346, -79.04715 34.303...",NaN
1,37,11620,02406252,3711620,Cerro Gordo,Cerro Gordo town,43,C1,N,N,G4110,A,1942481,0,+34.3228528,-078.9285494,"POLYGON ((-78.93988 34.32224, -78.93986 34.322...",NaN
2,37,11640,02406253,3711640,Chadbourn,Chadbourn town,43,C1,N,N,G4110,A,6821399,0,+34.3246297,-078.8254363,"POLYGON ((-78.79913 34.32875, -78.79911 34.328...",NaN
4,37,22080,02628624,3722080,Evergreen,Evergreen CDP,57,U1,N,N,G4210,S,9987201,0,+34.4148179,-078.9114303,"POLYGON ((-78.89626 34.42100, -78.88863 34.419...",NaN
5,37,06660,02405292,3706660,Boardman,Boardman town,43,C1,N,N,G4110,A,7953955,56896,+34.4300257,-078.9425233,"POLYGON ((-78.96351 34.43675, -78.96345 34.436...",NaN
10,37,06240,02405277,3706240,Bladenboro,Bladenboro town,43,C1,N,N,G4110,A,5737410,0,+34.5408702,-078.7946598,"MULTIPOLYGON (((-78.77172 34.55514, -78.77199 ...",NaN
12,37,09380,02402739,3709380,Butters,Butters CDP,57,U1,N,N,G4210,S,3401220,22873,+34.5600801,-078.8438636,"POLYGON ((-78.84365 34.56414, -78.84360 34.563...",NaN
21,37,37220,02404892,3737220,Laurinburg,Laurinburg city,25,C1,Y,N,G4110,A,32426895,411801,+34.7621569,-079.4773974,"POLYGON ((-79.51690 34.71199, -79.51681 34.712...",NaN
22,37,19620,02406414,3719620,East Laurinburg,East Laurinburg town,43,C1,N,N,G4110,A,490524,0,+34.7690768,-079.4450086,"POLYGON ((-79.44221 34.76669, -79.44245 34.766...",NaN
29,37,70480,02406815,3770480,Wagram,Wagram town,43,C1,N,N,G4110,A,3773706,0,+34.8890995,-079.3652077,"POLYGON ((-79.36822 34.89770, -79.36795 34.898...",NaN


## Add PUMA ID To Blocks
### Read in PUMA polygons for state and select places in study area

In [32]:
# Location of PUMA Polygons Defined By US Census
census_puma_shp = 'https://www2.census.gov/geo/tiger/TIGER2010/PUMA5/2010/tl_2010_37_puma10.zip'
census_puma_gdf = gpd.read_file(census_puma_shp)
census_puma_gdf.head()

,STATEFP10,PUMACE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,37,01302,3701302,Durham County (North)--Durham City (North) PUMA,G6120,S,488726893,28223974,+36.1044579,-078.8473651,"POLYGON ((-78.83869 35.96817, -78.83902 35.968..."
1,37,04100,3704100,"Lenoir, Onslow (North) & Jones Counties PUMA",G6120,S,3291200918,13262092,+35.0244080,-077.4813335,"POLYGON ((-77.82851 35.28106, -77.82842 35.282..."
2,37,03102,3703102,Charlotte City (Northwest) PUMA,G6120,S,146905291,936386,+35.2805783,-080.8733935,"POLYGON ((-80.98687 35.30113, -80.98657 35.301..."
3,37,01203,3701203,Wake County (Northeast)--Raleigh City (Northea...,G6120,S,227269722,1176477,+35.9307081,-078.4987418,"POLYGON ((-78.52670 36.01502, -78.52556 36.014..."
4,37,01801,3701801,Winston-Salem City (North) PUMA,G6120,S,168164470,1203335,+36.1381648,-080.2559043,"POLYGON ((-80.12744 36.14445, -80.12673 36.143..."


In [33]:
census_puma_gdf['GEOID10'].describe()

count          78
unique         78
top       3703105
freq            1
Name: GEOID10, dtype: object

In [34]:
census_puma_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [35]:
census_blocks_37155_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands.  British Virgin Island

In [36]:
# Find the bounds of the Census Block File
minx = census_blocks_37155_gdf.bounds.minx.min()
miny = census_blocks_37155_gdf.bounds.miny.min()
maxx = census_blocks_37155_gdf.bounds.maxx.max()
maxy = census_blocks_37155_gdf.bounds.maxy.max()
census_blocks_gdf_bounds = [minx, miny, maxx, maxy]
census_blocks_gdf_bounds

[-79.461754, 34.29924, -78.805144, 34.953637]

In [37]:
# Select pumas within Bounds of Study Area
# build the r-tree index - for pumas
sindex_census_puma_gdf = census_puma_gdf.sindex
possible_matches_index = list(sindex_census_puma_gdf.intersection(census_blocks_gdf_bounds))
area_census_puma_gdf = census_puma_gdf.iloc[possible_matches_index]
area_census_puma_gdf['GEOID10'].describe()

count           5
unique          5
top       3704900
freq            1
Name: GEOID10, dtype: object

In [38]:
# plot the intersections and the city
census_puma_gdf_map = fm.Map(location=[(miny+maxy)/2,(minx+maxx)/2], zoom_start=10)
fm.GeoJson(area_census_puma_gdf).add_to(census_puma_gdf_map)
display(census_puma_gdf_map)

### Spatial Join PUMA ID to Block IDS

In [39]:
# Confirm Count of Unique ID in layer to which data will be added
census_blocks_37155_gdf['BLOCKID10'].describe()

count                5799
unique               5799
top       371559613024004
freq                    1
Name: BLOCKID10, dtype: object

In [40]:
# build the r-tree index - Using Representative Point
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index,'geometry'] = census_blocks_37155_gdf['rppnt4269']
sindex_census_blocks_gdf = census_blocks_37155_gdf.sindex

# find the points that intersect with each subpolygon and add ID to Point
for index, puma in area_census_puma_gdf.iterrows():
    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex_census_blocks_gdf.intersection(puma['geometry'].bounds))
    possible_matches = census_blocks_37155_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(puma['geometry'])]
    census_blocks_37155_gdf.loc[precise_matches.index,'PUMGEOID10'] = puma['GEOID10']
    census_blocks_37155_gdf.loc[precise_matches.index,'PUMNAME10'] = puma['NAMELSAD10']

In [41]:
# Confirm Count of Unique ID in layer to which data will be added
census_blocks_37155_gdf['BLOCKID10'].describe()

count                5799
unique               5799
top       371559613024004
freq                    1
Name: BLOCKID10, dtype: object

In [42]:
census_blocks_37155_gdf['PUMGEOID10'].describe()

count        5799
unique          2
top       3705100
freq         5237
Name: PUMGEOID10, dtype: object

In [43]:
# Switch Block Geography back to polygons
census_blocks_37155_gdf.loc[census_blocks_37155_gdf.index,'geometry'] = census_blocks_37155_gdf['blk104269']
census_blocks_37155_gdf.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,CountySelect,...,blockid,tothupoints,popcount,HU100,POP100,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10
0,37,155,961900,2028,371559619002028,N,14,52,"POLYGON ((-79.22246 34.45884, -79.22253 34.458...",1,...,371559619002028,14,51,14,52,1,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA
1,37,155,961900,2054,371559619002054,N,1,3,"POLYGON ((-79.17985 34.40192, -79.18004 34.401...",1,...,371559619002054,1,3,1,3,0,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA
2,37,155,961700,2069,371559617002069,N,41,99,"POLYGON ((-79.17281 34.48092, -79.17275 34.480...",1,...,371559617002069,41,99,41,99,0,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA
3,37,155,961700,2065,371559617002065,N,6,22,"POLYGON ((-79.15764 34.50328, -79.15784 34.502...",1,...,371559617002065,6,21,6,22,1,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA
4,37,155,961700,2058,371559617002058,N,19,55,"POLYGON ((-79.15830 34.49735, -79.15664 34.498...",1,...,371559617002058,19,55,19,55,0,NaN,NaN,3705100,Robeson County (West)--Lumberton City PUMA


## Add School District ID To Blocks
### Read in School District polygons for state

The School Attendance Boundary Survey (SABS) was an experimental survey conducted by the National Center for Education Statistics (NCES) with assistance from the U.S. Census Bureau to collect school attendance boundaries for the 2013-2014 and 2015-2016 school years. The SABS collection includes boundaries for more than 70,000 schools in over 12,000 school districts throughout the U.S.

The SABS_1516 file is 556 MB. A previoud program selected the SABs for the County of Study.
The data has three overlapping layers - Hichschool, Middle School and Primary Schools. This section will add three new columns to the data one for each layer.

In [44]:
# Location of Unified School District Polygons Defined By US Census
sourcefolder = '../SourceData/nces.ed.gov/WorkNPR/'
sourceprogram = "NCES_2av1_SelectCountySchools_2021-06-06"
filename = sourcefolder+"/"+sourceprogram+"/"+"SABS_1516_37155_High.shp"
high_sabs_gdf = gpd.read_file(filename)
high_sabs_gdf.head()

,SrcName,ncessch,schnam,leaid,gslo,gshi,defacto,stAbbrev,openEnroll,Shape_Leng,Shape_Area,level,MultiBdy,slcncessch,slcleaid,geometry
0,Purnell Swett HS,370393002102,Purnell Swett High,3703930,09,12,0,NC,0,122991.586495,5.090427e+08,3,0,1,1,"POLYGON ((-79.21300 34.79015, -79.21441 34.779..."
1,S Robeson HS,370393002184,South Robeson High,3703930,09,12,0,NC,0,152761.549233,7.031522e+08,3,0,1,1,"POLYGON ((-79.28798 34.66740, -79.28630 34.666..."
2,Fairmont High,370393002232,Fairmont High,3703930,09,12,0,NC,0,150538.553654,7.662150e+08,3,0,1,1,"POLYGON ((-78.96800 34.55955, -78.96790 34.558..."
3,Lumberton HS,370393002237,Lumberton Senior High,3703930,09,12,0,NC,0,156694.540351,6.981158e+08,3,0,1,1,"POLYGON ((-78.84945 34.73266, -78.84932 34.732..."
4,Red Springs HS,370393002239,Red Springs High,3703930,09,12,0,NC,0,118402.139094,3.563852e+08,3,0,1,1,"POLYGON ((-79.07784 34.84129, -79.06428 34.833..."


In [45]:
high_sabs_gdf['ncessch'].describe()

count                6
unique               6
top       370393002232
freq                 1
Name: ncessch, dtype: object

In [46]:
# Save Work at this point as CSV
savefile = sys.path[0]+"/"+programname+"/"+programname+"EPSG4269.csv"
census_blocks_37155_gdf.to_csv(savefile)